In [ ]:
!pip install tensorflow-datasets

In [5]:
!pip install wandb

  Created wheel for pathtools: filename=pathtools-0.1.2-py3-none-any.whl size=8791 sha256=452ebbe498c2015f1a8b2a4549c287fe9e7a3176a46e85cefefeb1dc9ef793a3
  Stored in directory: /root/.cache/pip/wheels/e7/f3/22/152153d6eb222ee7a56ff8617d80ee5207207a8c00a7aab794
Successfully built pathtools


In [6]:
import os
import math
import wandb
import random
import numpy as np
import pandas as pd
import tensorflow as tf
import matplotlib.pyplot as plt

from tensorflow import keras
from tensorflow.keras.preprocessing.image import load_img

from tensorflow.keras.models import *
from tensorflow.keras.layers import *
from tensorflow.keras.losses import *
from tensorflow.keras.metrics import *
from tensorflow.keras.optimizers import *
from tensorflow.keras.activations import *

from tensorflow.keras.regularizers import *

from tensorflow.keras.callbacks import *
from tensorflow.keras.preprocessing import *

import tensorflow_datasets as tfds

dataset, info = tfds.load('oxford_iiit_pet:3.*.*', with_info=True)

Dl Completed...: 0 url [00:00, ? url/s]

Dl Size...: 0 MiB [00:00, ? MiB/s]

Extraction completed...: 0 file [00:00, ? file/s]

Generating splits...:   0%|          | 0/2 [00:00<?, ? splits/s]

Generating train examples...:   0%|          | 0/3680 [00:00<?, ? examples/s]

Shuffling /root/tensorflow_datasets/oxford_iiit_pet/3.2.0.incomplete0CE2R9/oxford_iiit_pet-train.tfrecord*...:…

Generating test examples...:   0%|          | 0/3669 [00:00<?, ? examples/s]

Shuffling /root/tensorflow_datasets/oxford_iiit_pet/3.2.0.incomplete0CE2R9/oxford_iiit_pet-test.tfrecord*...: …

Dataset oxford_iiit_pet downloaded and prepared to /root/tensorflow_datasets/oxford_iiit_pet/3.2.0. Subsequent calls will reuse this data.


In [7]:
def normalize(input_image, input_mask):
    # 배경 활성화
    input_image = tf.cast(input_image, tf.float32) / 255.0
    input_mask -= 1

    # 피사체라면 True로 반전.
    input_mask = 1 - (tf.cast(input_mask>=1, tf.float32) - tf.cast(input_mask>1, tf.float32))
    return input_image, input_mask


def load_image(datapoint):
    input_image = tf.image.resize(datapoint['image'], (128, 128))
    input_mask = tf.image.resize(datapoint['segmentation_mask'], (128,128))
    input_image, input_mask = normalize(input_image, input_mask)
    return input_image, input_mask

In [8]:
TRAIN_LENGTH = info.splits['train'].num_examples
BATCH_SIZE = 16
BUFFER_SIZE = 320
STEPS_PER_EPOCH = TRAIN_LENGTH // BATCH_SIZE

train_image = dataset['train'].map(load_image, num_parallel_calls=tf.data.AUTOTUNE)
test_image = dataset['test'].map(load_image, num_parallel_calls=tf.data.AUTOTUNE)

In [9]:
train_batches = (
    train_image
    .cache()
    .shuffle(BUFFER_SIZE)
    .batch(BATCH_SIZE)
    .prefetch(buffer_size=tf.data.AUTOTUNE)
)

test_batches = test_image.batch(BATCH_SIZE)

In [14]:
inputs = Input ((128, 128, 3))
x = Conv2D(64, 3, 1, padding='same', activation='relu') (inputs)
x = Conv2D(64, 3, 1, padding='same', activation='relu') (x)
x1 = x
X = MaxPooling2D()(x)


x = Conv2D(128, 3, 1, padding='same', activation='relu')(x)
x = Conv2D(128, 3, 1, padding='same', activation='relu')(x)
x2 = x
x = MaxPooling2D () (x)


x = Conv2D(192, 3, 1, padding='same', activation='relu')(x)
x = Conv2D(192, 3, 1, padding='same', activation='relu')(x)
x3 = x
x = MaxPooling2D()(x)


x = Conv2D(256, 3, 1, padding= 'same', activation='relu')(x)
x = Conv2D(256, 3, 1, padding= 'same', activation='relu')(x)
x4 = x
x = MaxPooling2D()(x)


x = Conv2D(384, 3, 1, padding='same', activation='relu')(x)
x = Conv2D(384, 3, 1, padding='same', activation='relu')(x)
x = UpSampling2D()(x)
x = Concatenate()([x4, x])


x = Conv2D(256, 3, 1, padding= 'same', activation='relu')(x)
x = Conv2D(256, 3, 1, padding= 'same', activation='relu')(x)
x = UpSampling2D()(x)
x = Concatenate()([x3, x])


x = Conv2D(192, 3, 1, padding= 'same', activation='relu')(x)
x = Conv2D(192, 3, 1, padding='same', activation='relu')(x)
x = UpSampling2D()(x)
x = Concatenate()([x2, x])


x = Conv2D(128, 3, 1, padding='same', activation='relu')(x)
x = Conv2D(128, 3, 1, padding='same', activation='relu')(x)
X = UpSampling2D()(x)
x = Concatenate()([x1, x])


x = Conv2D(64, 3, 1, padding='same', activation='relu')(x)
x = Conv2D(64, 3, 1, padding='same', activation='relu')(x)


outputs = Conv2D(1, 3, 1, padding='same', activation='sigmoid')(x) # 0(background), 1(foreground)
model = Model (inputs, outputs)

In [15]:
learning_rate = 1e-3 #0.001

def learning_rate_schedule(epoch):
    new_learning_rate = learning_rate

    if epoch <= 60:
        pass
    elif epoch > 60 and epoch <= 120:
        new_learning_rate = learning_rate * 0.1
    else:
        new_learning_rate = learning_rate * 0.01

    print('Learning rate:', new_learning_rate)

    return new_learning_rate


optimizer = tf.keras.optimizers.Adam(learning_rate=learning_rate)

In [ ]:
model.compile(optimizer=optimizer, loss='bce', metrics=['accuracy'])

from tensorflow.keras import callbacks
wandb.login(key='eed81e1c0a41dd8dd67a4ca90cea1be5a06d4eb0')
wandb.init(project="OxfordIIIT-Pet", entity="hcim", name='UNET')

wandbCallback=wandb.keras.WandbCallback(monitor="val_accuracy", mode="max",
                                        log_weights=True, log_gradients=True,
                                        training_data=train_batches, batch_size=BATCH_SIZE)


callbacks = [wandbCallback, LearningRateScheduler(learning_rate_schedule)]

model.fit(train_batches, validation_data=test_batches, epochs=200, workers=8, callbacks=callbacks)

wandb: W&B API key is configured. Use `wandb login --relogin` to force relogin
wandb: WARNING If you're specifying your api key in code, ensure this code is not shared publicly.
wandb: WARNING Consider setting the WANDB_API_KEY environment variable, or running `wandb login` from the command line.
wandb: Appending key for api.wandb.ai to your netrc file: /root/.netrc
wandb: Currently logged in as: chan4im (hcim). Use `wandb login --relogin` to force relogin


wandb: WARNING The save_model argument by default saves the model in the HDF5 format that cannot save custom objects like subclassed models and custom layers. This behavior will be deprecated in a future release in favor of the SavedModel format. Meanwhile, the HDF5 model is saved as W&B files and the SavedModel as W&B Artifacts.


Learning rate: 0.001
Epoch 1/200
  1/230 [..............................] - ETA: 4:58:09 - loss: 0.6900 - accuracy: 0.5828